In [2]:
import pandas as pd
import numpy as np
from math import exp, log, sqrt, pi, copysign
import functools
from scipy.stats import norm
sgn = functools.partial(copysign, 1)

# Load the Excel file
file_path = 'S_P500_ETF_Option_0917.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows to understand its structure
df.head()

,Calls,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Puts,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Strike,Ticker,Bid,Ask,Volm,Strike,Ticker,Bid,Ask,Volm
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2150,SPXW 9/29/17 C2150,259.299988,260.700012,0,2150,SPXW 9/29/17 P2150,34.799999,35.700001,10
3,2175,SPXW 9/29/17 C2175,238.100006,239.600006,0,2175,SPXW 9/29/17 P2175,38.5,39.400002,10
4,2200,SPXW 9/29/17 C2200,217.399994,218.899994,0,2200,SPXW 9/29/17 P2200,42.5,43.5,10


In [3]:
# Drop the second row which is NaN and reset the index
df_cleaned = df.drop(1).reset_index(drop=True)

# Renaming columns based on the first row and then drop the first row
df_cleaned.columns = ['Strike_Call', 'Ticker_Call', 'Bid_Call', 'Ask_Call', 'Volm_Call', 'Strike_Put', 'Ticker_Put', 'Bid_Put', 'Ask_Put', 'Volm_Put']
df_cleaned = df_cleaned.drop(0).reset_index(drop=True)

# Remove unnecessary columns ('Ticker', 'Volm')
df_cleaned = df_cleaned.drop(columns=['Ticker_Call', 'Volm_Call', 'Ticker_Put', 'Volm_Put'])

# Since Strike_Call and Strike_Put should be the same, we can check and then keep just one column for clarity
df_cleaned = df_cleaned.assign(Strike=df_cleaned['Strike_Call']).drop(columns=['Strike_Call', 'Strike_Put'])
# Calculating the mid price for calls and puts



# Drop any rows that contain NaN values
df_final = df_cleaned.dropna().reset_index(drop=True)
df_final 

for col in df_final.columns:
    # Check if the column should be numeric (this assumes you know which columns should be numeric)
    if df_final[col].dtype == 'object':
        # Convert column to numeric, set errors='coerce' to convert non-convertible values to NaN
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

df_final['Mid_Call'] = (df_final['Bid_Call']+ df_final['Ask_Call']) / 2
df_final['Mid_Put'] = (df_final['Bid_Put'] + df_final['Ask_Put']) / 2
df_final['C-P'] = df_final['Mid_Call'] - df_final['Mid_Put']
df_final.head()



,Bid_Call,Ask_Call,Bid_Put,Ask_Put,Strike,Mid_Call,Mid_Put,C-P
0,259.299988,260.700012,34.799999,35.700001,2150,260.000000,35.250000,224.750000
1,238.100006,239.600006,38.500000,39.400002,2175,238.850006,38.950001,199.900005
2,217.399994,218.899994,42.500000,43.500000,2200,218.149994,43.000000,175.149994
3,197.199997,198.699997,47.099998,48.099998,2225,197.949997,47.599998,150.349998
4,177.399994,178.899994,52.099998,53.200001,2250,178.149994,52.650000,125.499994


In [4]:
print(df_final.to_latex(float_format="%.2f", index=False, escape=True))

\begin{tabular}{rrrrrrrr}
\toprule
Bid\_Call & Ask\_Call & Bid\_Put & Ask\_Put & Strike & Mid\_Call & Mid\_Put & C-P \\
\midrule
259.30 & 260.70 & 34.80 & 35.70 & 2150 & 260.00 & 35.25 & 224.75 \\
238.10 & 239.60 & 38.50 & 39.40 & 2175 & 238.85 & 38.95 & 199.90 \\
217.40 & 218.90 & 42.50 & 43.50 & 2200 & 218.15 & 43.00 & 175.15 \\
197.20 & 198.70 & 47.10 & 48.10 & 2225 & 197.95 & 47.60 & 150.35 \\
177.40 & 178.90 & 52.10 & 53.20 & 2250 & 178.15 & 52.65 & 125.50 \\
158.30 & 159.70 & 57.80 & 58.90 & 2275 & 159.00 & 58.35 & 100.65 \\
139.80 & 141.30 & 64.10 & 65.30 & 2300 & 140.55 & 64.70 & 75.85 \\
122.10 & 123.60 & 71.20 & 72.50 & 2325 & 122.85 & 71.85 & 51.00 \\
105.30 & 106.80 & 79.10 & 80.50 & 2350 & 106.05 & 79.80 & 26.25 \\
89.50 & 90.90 & 88.10 & 89.50 & 2375 & 90.20 & 88.80 & 1.40 \\
74.80 & 76.10 & 98.10 & 99.50 & 2400 & 75.45 & 98.80 & -23.35 \\
61.30 & 62.60 & 109.40 & 110.90 & 2425 & 61.95 & 110.15 & -48.20 \\
49.10 & 50.30 & 122.00 & 123.50 & 2450 & 49.70 & 122.75 & -73.05 \

In [5]:
# Assuming df_final is already loaded and contains 'Strike' and 'C-P'
# Prepare the matrix A and vector y using real data
A = np.vstack([np.ones(len(df_final)), -df_final['Strike'].values]).T
y = df_final['C-P'].values

# Using the normal equation to solve for the parameters
x = np.linalg.inv(A.T @ A) @ A.T @ y

# Output the estimated values for demonstration
print("Estimated S * e^(-qT):", x[0])
print("Estimated e^(-rT):", x[1])


Estimated S * e^(-qT): 2360.1743583417406
Estimated e^(-rT): 0.993169427730553


In [6]:
from datetime import datetime

# Dates for calculation
start_date = datetime(2017, 3, 16)
end_date = datetime(2017, 9, 29)

# Calculate the number of days between the two dates
delta = end_date - start_date

# Convert days to years

print(delta.days)
print(delta.days / 365.25)
print(142/260)  # ChatGPT
print(138/252)  # ChatGPT

T = 138/252  # Kevin

197
0.5393566050650239
0.5461538461538461
0.5476190476190477


In [7]:
import numpy as np

# Constants
S_e_minus_qT = 2360.1743583417406
e_minus_rT = 0.993169427730553
S = 2381

# Calculate q and r
q = -np.log(S_e_minus_qT / S) / T
r = -np.log(e_minus_rT) / T

q, r

(0.016042302730637618, 0.012516013524197112)

In [8]:
import pandas as pd
from math import exp, log, sqrt, pi
from scipy.stats import norm

# Constants
PVF: float = 2360.1743583417406  # Present Value Factor: S * e^(-qT)
disc: float = 0.993169427730553  # Discount Factor: e^(-rT)
T: float = 138./252  # Time to Expiration in years

# Black-Scholes related functions
def d1(K: float, sigma: float) -> float:
    """ Calculate d1 for the Black-Scholes formula. """
    return (log(PVF / (K * disc)) + (sigma**2 / 2) * T) / (sigma * sqrt(T))

def d2(K: float, sigma: float) -> float:
    """ Calculate d2 for the Black-Scholes formula, dependent on d1. """
    return d1(K, sigma) - sigma * sqrt(T)

def black_scholes_call_price(K: float, sigma: float) -> float:
    """ Calculate the Black-Scholes price of a call option. """
    return PVF * norm.cdf(d1(K, sigma)) - K * disc * norm.cdf(d2(K, sigma))

def black_scholes_put_price(K: float, sigma: float) -> float:
    """ Calculate the Black-Scholes price of a call option. """
    return  K * disc * norm.cdf(-d2(K, sigma)) - PVF * norm.cdf(-d1(K, sigma))

def derivative_f(K: float, sigma: float) -> float:
    """ Calculate the derivative of the call price with respect to sigma (Vega). """
    return PVF * sqrt(T / (2 * pi)) * exp(-(d1(K, sigma)**2) / 2)

def normalized_call_price_difference(K: float, sigma: float, market_price: float) -> float:
    """ Calculate the normalized difference between the implied and market price. """
    return (black_scholes_call_price(K, sigma) - market_price) / derivative_f(K, sigma)

def normalized_put_price_difference(K: float, sigma: float, market_price: float) -> float:
    """ Calculate the normalized difference between the implied and market price. """
    return (black_scholes_put_price(K, sigma) - market_price) / derivative_f(K, sigma)

# Initial conditions
sigma_start: float = 0.25
K_example: float = 2150
market_price_example: float = 260

# Iteration to adjust sigma
results = []
current_sigma = sigma_start
norm_diff_call = normalized_call_price_difference(K_example, current_sigma, market_price_example)

while abs(norm_diff_call) > 1e-7:
    # Calculate all values and store in the results
    d1_val = d1(K_example, current_sigma)
    d2_val = d2(K_example, current_sigma)
    call_price = black_scholes_call_price(K_example, current_sigma)
    vega_C = derivative_f(K_example, current_sigma)
    
    # Store the iteration results
    results.append({
        "sigma": current_sigma,
        "d1": d1_val,
        "d2": d2_val,
        "C_BS (Call Price)": call_price,
        "vega_C": vega_C,
        "delta_x": norm_diff_call
    })

    # Update sigma and recalculate the normalized difference
    current_sigma -= norm_diff_call
    norm_diff_call = normalized_call_price_difference(K_example, current_sigma, market_price_example)

print(current_sigma)

# Convert results to DataFrame for display
df = pd.DataFrame(results)
print(df.to_latex(escape=True))


0.17203247131575483
\begin{tabular}{lrrrrrr}
\toprule
 & sigma & d1 & d2 & C\_BS (Call Price) & vega\_C & delta\_x \\
\midrule
0 & 0.250000 & 0.633690 & 0.448687 & 301.682311 & 570.026470 & 0.073123 \\
1 & 0.176877 & 0.830370 & 0.699479 & 262.371538 & 493.591095 & 0.004805 \\
2 & 0.172072 & 0.849951 & 0.722615 & 260.019132 & 485.537459 & 0.000039 \\
\bottomrule
\end{tabular}



In [9]:
def get_implied_vol(K: float, mp: float, diff_calculator_func, sigma_start=0.25):
    current_sigma = sigma_start
    norm_diff = diff_calculator_func(K, current_sigma, mp)

    while abs(norm_diff) > 1e-6:
        current_sigma -= norm_diff
        norm_diff = diff_calculator_func(K, current_sigma, mp)
    
    return current_sigma

In [10]:
get_implied_vol(2150, 260, normalized_call_price_difference)

0.17203247131575483

In [11]:
get_implied_vol(2150, 35.25, normalized_put_price_difference)

0.17225914724147992

In [12]:
# Assume df_final exists with columns 'Strike' and 'Mid_Call'
# df_final = pd.DataFrame({'Strike': [2150, 2200, 2250], 'Mid_Call': [100, 105, 110]})  # Example structure

results = []

# Iterate over each row in the DataFrame
for index, row in df_final.iterrows():
    K = row['Strike']
    market_price_call = row['Mid_Call']
    market_price_put = row['Mid_Put']
    # print(K, market_price_call, market_price_put)

    implied_call = get_implied_vol(K, market_price_call, normalized_call_price_difference)
    implied_put = get_implied_vol(K, market_price_put, normalized_put_price_difference)
    
    # Store the final sigma and other details
    results.append({
        "Strike Price": K,
        "Impl. Call Vol.": implied_call,
        "Impl. Put Vol.": implied_put
    })

# Convert results to DataFrame for display
implied_vol_df = pd.DataFrame(results)
print(implied_vol_df)

    Strike Price  Impl. Call Vol.  Impl. Put Vol.
0         2150.0         0.172032        0.172259
1         2175.0         0.167198        0.167452
2         2200.0         0.162417        0.162512
3         2225.0         0.157677        0.157716
4         2250.0         0.152705        0.152776
5         2275.0         0.147821        0.147923
6         2300.0         0.142977        0.143030
7         2325.0         0.138117        0.138199
8         2350.0         0.133327        0.133293
9         2375.0         0.128535        0.128530
10        2400.0         0.123806        0.123688
11        2425.0         0.119208        0.119119
12        2450.0         0.114600        0.114539
13        2475.0         0.110395        0.110204
14        2500.0         0.106311        0.106140
15        2550.0         0.099073        0.097949
16        2600.0         0.094041        0.092220
17        2650.0         0.091325        0.093151
18        2700.0         0.091060        0.093877


In [13]:
df_final["impl call vol"] = implied_vol_df["Impl. Call Vol."]
df_final["impl put vol"] = implied_vol_df["Impl. Put Vol."]
df_final["impl vol rel diff [%]"] = abs(df_final["impl call vol"] - df_final["impl put vol"])/df_final["impl put vol"]*100
print(df_final[["Strike", "Mid_Call", "Mid_Put", "impl call vol", "impl put vol","impl vol rel diff [%]"]].to_latex(index=False, float_format="%.3f", escape=True))

\begin{tabular}{rrrrrr}
\toprule
Strike & Mid\_Call & Mid\_Put & impl call vol & impl put vol & impl vol rel diff [\%] \\
\midrule
2150 & 260.000 & 35.250 & 0.172 & 0.172 & 0.132 \\
2175 & 238.850 & 38.950 & 0.167 & 0.167 & 0.152 \\
2200 & 218.150 & 43.000 & 0.162 & 0.163 & 0.058 \\
2225 & 197.950 & 47.600 & 0.158 & 0.158 & 0.025 \\
2250 & 178.150 & 52.650 & 0.153 & 0.153 & 0.047 \\
2275 & 159.000 & 58.350 & 0.148 & 0.148 & 0.069 \\
2300 & 140.550 & 64.700 & 0.143 & 0.143 & 0.037 \\
2325 & 122.850 & 71.850 & 0.138 & 0.138 & 0.060 \\
2350 & 106.050 & 79.800 & 0.133 & 0.133 & 0.026 \\
2375 & 90.200 & 88.800 & 0.129 & 0.129 & 0.003 \\
2400 & 75.450 & 98.800 & 0.124 & 0.124 & 0.096 \\
2425 & 61.950 & 110.150 & 0.119 & 0.119 & 0.075 \\
2450 & 49.700 & 122.750 & 0.115 & 0.115 & 0.054 \\
2475 & 39.050 & 136.850 & 0.110 & 0.110 & 0.173 \\
2500 & 29.850 & 152.500 & 0.106 & 0.106 & 0.161 \\
2550 & 16.050 & 187.950 & 0.099 & 0.098 & 1.147 \\
2600 & 7.950 & 229.450 & 0.094 & 0.092 & 1.974 \\
2650 

In [14]:
F = S_e_minus_qT * 1/e_minus_rT
F2 = S*exp((r-q)*T)
print(S_e_minus_qT, e_minus_rT, F, pi, r, T, F2)

2360.1743583417406 0.993169427730553 2376.4065751952007 3.141592653589793 0.012516013524197112 0.5476190476190477 2376.4065751952003


In [15]:
# S_e_minus_qT = 2360.1743583417406
# e_minus_rT = 0.993169427730553
# S = 2381
# T= 138/252

# q = -log(S_e_minus_qT / S) / T
# r = -log(e_minus_rT) / T

# F = S_e_minus_qT * 1/e_minus_rT

df = df_final[["Strike", "Mid_Call", "Mid_Put", "impl call vol", "impl put vol"]].copy()

df["y"] = np.log(F/df["Strike"])
df["alpha_C"] = df["Mid_Call"]/(df["Strike"] * exp(-r*T))
df["R_C"] = 2*df["alpha_C"] - np.exp(df["y"]) + 1
df["B_C"] = 4 * (np.exp((2/pi)*df["y"]) + np.exp(-(2/pi)*df["y"])) - 2*np.exp(-df["y"])*(np.exp((1-2/pi)*df["y"]) + np.exp(-(1-2/pi)*df["y"])) * (np.exp(2*df["y"]) + 1 - np.power(df["R_C"],2))
df["C_C"] = np.exp(-2*df["y"]) * (np.power(df["R_C"],2) - np.power(np.exp(df["y"]) - 1,2)) * (np.power(np.exp(df["y"]) + 1,2) - np.power(df["R_C"],2))

df["A"] = np.power(np.exp((1-2/pi)*df["y"]) - np.exp(-(1-2/pi)*df["y"]),2)

df["alpha_P"] = df["Mid_Put"]/(df["Strike"] * exp(-r*T))
df["R_P"] = 2*df["alpha_P"] + np.exp(df["y"]) - 1
df["B_P"] = 4*(np.exp((2/pi)*df["y"]) + np.exp(-(2/pi)*df["y"])) - 2*np.exp(-df["y"])*(np.exp((1-2/pi)*df["y"]) + np.exp(-(1-2/pi)*df["y"])) * (np.exp(2*df["y"]) + 1 - np.power(df["R_P"],2))
df["C_P"] = np.exp(-2*df["y"]) * (np.power(df["R_P"],2) - np.power(np.exp(df["y"]) - 1,2)) * (np.power(np.exp(df["y"]) + 1,2) - np.power(df["R_P"],2))

df["beta_C"] = 2 * df["C_C"] / (df["B_C"] + np.sqrt(np.power(df["B_C"],2) + 4*df["A"]*df["C_C"]))
df["gamma_C"] = -pi/2*np.log(df["beta_C"])

df["beta_P"] = 2 * df["C_P"] / (df["B_P"] + np.sqrt(np.power(df["B_P"],2) + 4*df["A"]*df["C_P"]))
df["gamma_P"] = -pi/2*np.log(df["beta_P"])



def A(x: float) -> float:
    return 1/2 + sgn(x)/2 * sqrt(1 - exp(-2*x*x/pi))

df["CP_00"] = df["Strike"]*np.exp(-r*T)

def calculate_sigma_c(row):
    g = row["gamma_C"]
    y = row["y"]
    if y >= 0:
        C_0 = row["CP_00"] * (exp(y) * A(sqrt(2*y)) - 1/2)
        if row['Mid_Call'] <= C_0:
            return 1 / sqrt(T) * (sqrt(g+y) - sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))
    else:
        C_0 = row["CP_00"] * (exp(y)/2 - A(-sqrt(-2*y)))
        if row['Mid_Call'] <= C_0:
            return 1 / sqrt(T) * (-sqrt(g+y) + sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))

def calculate_sigma_p(row):
    g = row["gamma_P"]
    y = row["y"]
    if y >= 0:
        P_0 = row["CP_00"] * (1/2 - exp(y) * A(-sqrt(2*y)))
        if row['Mid_Call'] <= P_0:
            return 1 / sqrt(T) * (sqrt(g+y) - sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))
    else:
        P_0 = row["CP_00"] * (A(sqrt(-2*y)) - exp(y)/2)
        if row['Mid_Call'] <= P_0:
            return 1 / sqrt(T) * (-sqrt(g+y) + sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))

# Apply calculation to DataFrame
df['sigma_C'] = df.apply(calculate_sigma_c, axis=1)
df['sigma_P'] = df.apply(calculate_sigma_p, axis=1)

print(df.to_latex(index=False, float_format="%.2f", escape=True))


\begin{tabular}{rrrrrrrrrrrrrrrrrrrrrr}
\toprule
Strike & Mid\_Call & Mid\_Put & impl call vol & impl put vol & y & alpha\_C & R\_C & B\_C & C\_C & A & alpha\_P & R\_P & B\_P & C\_P & beta\_C & gamma\_C & beta\_P & gamma\_P & CP\_00 & sigma\_C & sigma\_P \\
\midrule
2150 & 260.00 & 35.25 & 0.17 & 0.17 & 0.10 & 0.12 & 0.14 & 0.04 & 0.03 & 0.01 & 0.02 & 0.14 & 0.04 & 0.03 & 0.67 & 0.64 & 0.67 & 0.64 & 2135.31 & 0.17 & 0.17 \\
2175 & 238.85 & 38.95 & 0.17 & 0.17 & 0.09 & 0.11 & 0.13 & 0.04 & 0.03 & 0.00 & 0.02 & 0.13 & 0.04 & 0.03 & 0.71 & 0.53 & 0.72 & 0.53 & 2160.14 & 0.17 & 0.17 \\
2200 & 218.15 & 43.00 & 0.16 & 0.16 & 0.08 & 0.10 & 0.12 & 0.04 & 0.03 & 0.00 & 0.02 & 0.12 & 0.04 & 0.03 & 0.76 & 0.42 & 0.76 & 0.42 & 2184.97 & 0.16 & 0.16 \\
2225 & 197.95 & 47.60 & 0.16 & 0.16 & 0.07 & 0.09 & 0.11 & 0.03 & 0.03 & 0.00 & 0.02 & 0.11 & 0.03 & 0.03 & 0.81 & 0.33 & 0.81 & 0.33 & 2209.80 & 0.16 & 0.16 \\
2250 & 178.15 & 52.65 & 0.15 & 0.15 & 0.05 & 0.08 & 0.10 & 0.03 & 0.03 & 0.00 & 0.02 & 0.

In [18]:
df[df["Strike"] == 2550][["Strike","A", "B_C", "C_C", "B_P", "C_P"]]

,Strike,A,B_C,C_C,B_P,C_P
15,2550,0.002626,0.013533,0.008092,0.013256,0.007816


In [15]:
K = 2150
C_m = 260

def compute_implied_volatility(F, K, C_m, r, T):
    # Calculate necessary variables
    y = log(F / K)
    alpha_C = C_m / (K * exp(-r * T))
    R = 2 * alpha_C - exp(y) + 1
    
    # Display intermediate variables
    print(f"y = {y}")
    print(f"alpha_C = {alpha_C}")
    print(f"R = {R}")

    # Calculate A, B, C using the correct formulas
    A = (exp((1 - 2 / pi) * y) - exp(-(1 - 2 / pi) * y))**2
    B = 4 * (exp(2 / pi * y) + exp(-2 / pi * y)) - 2 * exp(-y) * (exp((1 - 2 / pi) * y) + exp(-(1 - 2 / pi) * y)) * (exp(2 * y) + 1 - R**2)
    C = exp(-2 * y) * (R**2 - (exp(y) - 1)**2) * ((exp(y) + 1)**2 - R**2)

    # Display A, B, C
    print(f"A = {A}")
    print(f"B = {B}")
    print(f"C = {C}")

    # Calculate β and γ
    beta = 2 * C / (B + sqrt(B**2 + 4 * A * C))
    print(f"beta = {beta}")
    gamma = -pi / 2 * log(beta)
    
    # Display β and γ
    print(f"beta = {beta}")
    print(f"gamma = {gamma}")

    return gamma

# Example usage with hypothetical values

gamma_value = compute_implied_volatility(F, K, C_m, r, T)
print("Calculated Gamma value:", gamma_value)


y = 0.10012166222829415
alpha_C = 0.12176193626345487
R = 0.1382184887151885
A = 0.005297003878490904
B = 0.039986327424356816
C = 0.028953432955530163
beta = 0.6654264542268135
beta = 0.6654264542268135
gamma = 0.6398280056475636
Calculated Gamma value: 0.6398280056475636


In [16]:
def A_func(x):
    # Compute A based on a function of x
    return 1/2 + sgn(x)/2 * sqrt(1 - exp(-2*x*x/pi))

def calculate_sigma(F, K, C_m, r, T, gamma):
    y = log(F / K)
    A_value = A_func(sqrt(2 * y)) if y >= 0 else A_func(-sqrt(-2 * y))
    
    if y >= 0:
        # Corrected the formula for C0 where gamma is not in the exponent, but y is
        C0 = K * exp(-r * T) * (exp(y) * A_value - 0.5)
        if C_m <= C0:
            sigma = 1 / sqrt(T) * (sqrt(gamma + y) - sqrt(gamma - y))
        else:
            sigma = 1 / sqrt(T) * (sqrt(gamma + y) + sqrt(gamma - y))
    else:
        # Corrected the usage of A_value and y in the formula for C0
        C0 = K * exp(-r * T) * (exp(y)/2 - A_value)
        if C_m <= C0:
            sigma = 1 / sqrt(T) * (-sqrt(gamma + y) + sqrt(gamma - y))
        else:
            sigma = 1 / sqrt(T) * (sqrt(gamma + y) + sqrt(gamma - y))
    
    return sigma

sigma_value = calculate_sigma(F, K, C_m, r, T, gamma_value)
print("Calculated Sigma value:", sigma_value)


Calculated Sigma value: 0.16966756754181977


In [17]:
df["call vol diff [%]"] = (df["sigma_C"] - df["impl call vol"])/df["impl call vol"]*100
df["put vol diff [%]"] = (df["sigma_P"] - df["impl put vol"])/df["impl put vol"]*100
print(df[["Strike", "impl call vol", "impl put vol", "sigma_C", "sigma_P", "call vol diff [%]", "put vol diff [%]"]].to_latex(escape=True, index=False, float_format="%.3f"))

\begin{tabular}{rrrrrrr}
\toprule
Strike & impl call vol & impl put vol & sigma\_C & sigma\_P & call vol diff [\%] & put vol diff [\%] \\
\midrule
2150 & 0.172 & 0.172 & 0.170 & 0.170 & -1.375 & -1.371 \\
2175 & 0.167 & 0.167 & 0.165 & 0.166 & -1.144 & -1.141 \\
2200 & 0.162 & 0.163 & 0.161 & 0.161 & -0.923 & -0.922 \\
2225 & 0.158 & 0.158 & 0.157 & 0.157 & -0.716 & -0.716 \\
2250 & 0.153 & 0.153 & 0.152 & 0.152 & -0.528 & -0.527 \\
2275 & 0.148 & 0.148 & 0.147 & 0.147 & -0.360 & -0.359 \\
2300 & 0.143 & 0.143 & 0.143 & 0.143 & -0.217 & -0.217 \\
2325 & 0.138 & 0.138 & 0.138 & 0.138 & -0.105 & -0.105 \\
2350 & 0.133 & 0.133 & 0.133 & 0.133 & -0.031 & -0.031 \\
2375 & 0.129 & 0.129 & 0.129 & 0.129 & -0.002 & -0.002 \\
2400 & 0.124 & 0.124 & 0.124 & 0.124 & -0.028 & -0.028 \\
2425 & 0.119 & 0.119 & 0.119 & 0.119 & -0.120 & -0.120 \\
2450 & 0.115 & 0.115 & 0.114 & 0.114 & -0.292 & -0.293 \\
2475 & 0.110 & 0.110 & 0.110 & 0.110 & -0.557 & -0.559 \\
2500 & 0.106 & 0.106 & 0.105 & 0.105 & -0